<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [12]</a>'.</span>

# Joint multitask

The point of this notebook is not to do anything useful, but to show what is possible with relatively low effort using the *transformer_heads* library. In this example, we will train four heads on a transformer model while using qlora to finetune the transformer block weights. The first head will be hooked at layer 9 (-4) and predict the sentiment of imdb reviews (text classification). The second head will be hooked at the last layer (-1 or 12) and does causal language modelling on imdb reviews. The third head will be hooked at layer 6 (-7) and will learn to count the number of occurences of each letter of the alphabet occuring in imdb reviews (Text-level regression). The final head will be hooked at layer 4 (-9) and will predict how many tokens will follow before the review ends for each token in imdb reviews (Token-level regression). The final head will also be a small mlp instead of a linear head.

All heads and the qlora parameters will be trained jointly (multi-task learning).

In [1]:
from transformer_heads import create_headed_qlora, load_lora_with_heads
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    MistralForCausalLM,
    Trainer,
    BitsAndBytesConfig,
    TrainingArguments,
    GPT2Model,
    GPT2LMHeadModel,
)
from transformer_heads.util.helpers import DataCollatorWithPadding, get_model_params
from peft import LoraConfig
from transformer_heads.config import HeadConfig
from transformer_heads.util.model import print_trainable_parameters
from transformer_heads.util.evaluate import (
    evaluate_head_wise,
)
import torch
import pandas as pd

In [2]:
model_path = "gpt2"
train_epochs = 1
eval_epochs = 1
logging_steps = 100

In [3]:
# Parameters
model_path = "mistralai/Mistral-7B-v0.1"
train_epochs = 1
eval_epochs = 1
logging_steps = 100


In [4]:
model_params = get_model_params(model_path)
model_class = model_params["model_class"]
hidden_size = model_params["hidden_size"]
vocab_size = model_params["vocab_size"]
print(model_params)

{'model_class': <class 'transformers.models.mistral.modeling_mistral.MistralForCausalLM'>, 'hidden_size': 4096, 'vocab_size': 32000}


Define the various different heads. Given the differences in loss functions and magnitudes of label data in the dataset, it is important to weigh the losses of each head so that training is given similar importance for all of them.

In [5]:
head_configs = [
    HeadConfig(
        name=f"sentiment_head",
        layer_hook=-4,
        in_size=hidden_size,
        output_activation="linear",
        pred_for_sequence=True,
        loss_fct="cross_entropy",
        num_outputs=2,
        loss_weight=2.0,
    ),
    HeadConfig(
        name=f"causal_lm",
        layer_hook=-1,
        in_size=hidden_size,
        output_activation="linear",
        is_causal_lm=True,
        loss_fct="cross_entropy",
        num_outputs=vocab_size,
        is_regression=False,
        output_bias=False,
        loss_weight=1.0,
    ),
    HeadConfig(
        name=f"alphabet_regression",
        layer_hook=-7,
        in_size=hidden_size,
        output_activation="linear",
        is_causal_lm=False,
        pred_for_sequence=True,
        loss_fct="mse",
        num_outputs=26,  # 26 letters in the alphabet
        is_regression=True,
        loss_weight=0.002,
    ),
    HeadConfig(
        name=f"num_tokens_regression",
        layer_hook=-7,
        hidden_size=128,  # MLP hidden size
        num_layers=3,  # 2 hidden layers in MLP
        in_size=hidden_size,
        output_activation="linear",
        is_causal_lm=False,
        pred_for_sequence=False,
        loss_fct="mse",
        num_outputs=1,
        is_regression=True,
        loss_weight=0.0002,
    ),
    HeadConfig(
        name=f"lm_head",  # Let's also keep the original lm head for comparison
        layer_hook=-1,
        in_size=hidden_size,
        output_activation="linear",
        is_causal_lm=True,
        pred_for_sequence=False,
        loss_fct="cross_entropy",
        num_outputs=vocab_size,
        is_regression=False,
        trainable=False,  # Keep it in it's pretrained state
    ),
]

In [6]:
dd = load_dataset("imdb")

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token


def processing_function(examples):
    out = tokenizer(examples["text"], padding=False, truncation=True)
    out["sentiment_head"] = examples["label"]
    out["causal_lm"] = out["lm_head"] = out["input_ids"].copy()
    out["num_tokens_regression"] = [
        list(map(float, range(len(ids) - 1, -1, -1))) for ids in out["input_ids"]
    ]
    out["alphabet_regression"] = [
        [
            float(text.count(x) + text.count(x.upper()))
            for x in "abcdefghijklmnopqrstuvwxyz"
        ]
        for text in examples["text"]
    ]
    return out


for split in dd.keys():
    dd[split] = dd[split].filter(function=lambda example: len(example["text"]) > 10)
    dd[split] = dd[split].shuffle()
    dd[split] = dd[split].map(processing_function, batched=True)

dd.set_format(
    type="torch",
    columns=["input_ids", "attention_mask"] + [x.name for x in head_configs],
)
for split in dd.keys():
    dd[split] = dd[split].remove_columns(["text", "label"])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
dd["test"]

Dataset({
    features: ['input_ids', 'attention_mask', 'sentiment_head', 'causal_lm', 'lm_head', 'num_tokens_regression', 'alphabet_regression'],
    num_rows: 25000
})

Setting *target_modules=None* in the qlora config will make *create_headed_qlora* create LoRA modules for all linear layers in the transformer.

In [9]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_in_8bit=False,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_compute_dtype=torch.float32,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)
lora_config = LoraConfig(
    r=32,
    lora_alpha=16,
    target_modules=None,
    lora_dropout=0.0,
    bias="none",
    task_type="CAUSAL_LM",
)
model = create_headed_qlora(
    base_model_class=model_class,
    model_name=model_path,
    quantization_config=quantization_config,
    lora_config=lora_config,
    head_configs=head_configs,
    fully_trained_heads=True,
    device_map={"": torch.cuda.current_device()},
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of TransformerWithHeads were not initialized from the model checkpoint at mistralai/Mistral-7B-v0.1 and are newly initialized: ['heads.alphabet_regression.lins.0.weight', 'heads.causal_lm.lins.0.weight', 'heads.num_tokens_regression.lins.0.bias', 'heads.num_tokens_regression.lins.0.weight', 'heads.num_tokens_regression.lins.1.bias', 'heads.num_tokens_regression.lins.1.weight', 'heads.num_tokens_regression.lins.2.weight', 'heads.sentiment_head.lins.0.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
print_trainable_parameters(model)

all params: 3967684992 || trainable params: 215613824 || trainable%: 5.434247538167465
params by dtype: defaultdict(<class 'int'>, {torch.float32: 478024064, torch.uint8: 3489660928})
trainable params by dtype: defaultdict(<class 'int'>, {torch.float32: 215613824})


In [11]:
collator = DataCollatorWithPadding(
    feature_name_to_padding_value={
        "input_ids": tokenizer.pad_token_id,
        "attention_mask": 0,
        "causal_lm": -100,
        "lm_head": -100,
        "num_tokens_regression": 0,
    }
)

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [12]:
print(evaluate_head_wise(model, dd["test"], collator, epochs=eval_epochs))

Evaluating:   0%|          | 0/3125 [00:00<?, ?it/s]

Evaluating:   0%|          | 1/3125 [00:04<3:54:04,  4.50s/it]

Evaluating:   0%|          | 2/3125 [00:10<4:49:00,  5.55s/it]

Evaluating:   0%|          | 3/3125 [00:15<4:38:54,  5.36s/it]

Evaluating:   0%|          | 4/3125 [00:19<4:05:01,  4.71s/it]

...

Evaluating:   9%|▉         | 291/3125 [26:52<4:51:23,  6.17s/it]

Evaluating:   9%|▉         | 292/3125 [26:55<4:03:59,  5.17s/it]

Evaluating:   9%|▉         | 293/3125 [26:59<3:55:47,  5.00s/it]

Evaluating:   9%|▉         | 293/3125 [27:24<4:24:58,  5.61s/it]

OutOfMemoryError: CUDA out of memory. Tried to allocate 3.01 GiB. GPU 0 has a total capacity of 39.39 GiB of which 255.06 MiB is free. Including non-PyTorch memory, this process has 39.13 GiB memory in use. Of the allocated memory 34.75 GiB is allocated by PyTorch, and 3.89 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
args = TrainingArguments(
    output_dir="imdb_linear_probe",
    learning_rate=0.0002,
    num_train_epochs=train_epochs,  # To speed things up set to 0.1, set to 1 for better performance
    logging_steps=logging_steps,
    do_eval=False,
    remove_unused_columns=False,
)
trainer = Trainer(
    model,
    args=args,
    train_dataset=dd["train"],
    data_collator=collator,
)
trainer.train()

In [ ]:
evals = evaluate_head_wise(model, dd["test"], collator, epochs=eval_epochs)
print(evals)

# Saving and loading
Now let's how to save a complicated mulit-headed model to then load it again for inference. Saving is super easy. Just call save_pretrained and all trained parameters will be saved correctly. The saving will also work correctly for checkpoints created during training.

In [ ]:
model.save_pretrained("qlora_multitask_imdb")
del model

While loading the model, we need to make sure to correctly attach and initialize all the heads, so that won't easily work with the huggingface api. Instead, *transformer_heads* provides the *load_lora_with_heads* function. Note that giving a quantization config is optional here. We could also give a different quantization config or none at all.

In [ ]:
model = load_lora_with_heads(
    model_class,
    "qlora_multitask_imdb",
    quantization_config,
    device_map={"": torch.cuda.current_device()},
)

Let's now find out if the loaded model behaves the same as the saved model:

In [ ]:
new_evals = evaluate_head_wise(model, dd["test"], collator, epochs=eval_epochs)
print(new_evals)